# Multi Agent orchestrator using Azure AI Agentic Service

**Problem Statement**: AI agents are autonomous software entities designed to perform tasks, make decisions, and interact with environments using artificial intelligence, machine learning, natural language processing, and reinforcement learning. However, every custom AI agent written today needs lifecycle management, that includes packaging as containers, deployment, scaling, allocation of right resources etc. As the number of agents grow in your ecosystem it becomes tedious to manage the environment.

**Azure AI Agent Service** is a fully managed service designed to empower developers to securely build, deploy, and scale high-quality, and extensible AI agents without needing to manage the underlying compute and storage resources. We can use Azure AI Agent Service to create and run an agent in just a few lines of code. You can also manage complex workflows with **AutoGen** and **Semantic Kernel**.

![AzureAIAgentService](../images/azure-agentic-service.png)

In this notebook, I will show the capabilities of **Azure AI Agent Service** along with add-ons like Bing Service, Tools. At the end of this tutorial we will summarize a research paper into LinkedIn post using Azure Agentic Service, Bing & LLMs.

**Pre-requisites**
- Azure Subscription
- Azure Open AI
- Azure AI Foundry Project ([Link](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/create-projects?tabs=ai-studio))

**ENV Config**
- `PROJECT_CONNECTION_STRING`: Connection string for the Azure AI project.
- `SEARCH_KEY`: Key for the Azure AI search service.
- `SEARCH_ENDPOINT`: Endpoint for the Azure AI search service.
- `AOAI_ENDPOINT`: Endpoint for the Azure OpenAI service.
- `AOAI_KEY`: Key for the Azure OpenAI service.


In [ ]:
## Project PIP requirements
# %pip install -q azure-ai-projects azure-identity azure-ai-ml azure-search-documents tika "autogen-agentchat" "autogen-ext[openai]"

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.models import UserMessage


# Load environment variables from .env file
load_dotenv()

# Research paper path
research_paper_path =  "https://arxiv.org/pdf/2503.05142"

Before proceeding make sure you have created an Azure AI Agentic Service project. Copy the connection string to the .env file.

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

conn_list = project_client.connections._list_connections()["value"]
conn_id = ""

# Search in the metadata field of each connection in the list for the azure_ai_search type and get the id value to establish the variable
for conn in conn_list:
    metadata = conn["properties"].get("metadata", {})
    if metadata.get("type", "").upper() == "AZURE_AI_SEARCH":
        conn_id = conn["id"]
        break
    
print(f"Connection ID: {conn_id}")

The below model will be used for orchestration, you may choose any Open AI model from Azure Open AI.

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    api_version="2024-05-01-preview",
    model = "gpt-4o-mini",
    azure_endpoint=os.environ["AOAI_ENDPOINT"], # Azure OpenAI endpoint.
    api_key=os.environ["AOAI_KEY"], # For key-based authentication.
)

# Test the Azure OpenAI model client
result = await az_model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)


1. Create Azure AI Search and configure as connector to the agent. [Link](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=pythonsdk%2Cpython&pivots=code-examples#setup-create-an-agent-that-can-use-an-existing-azure-ai-search-index)

### Download and ingest the file to vector store

In the below step we are downloading the research paper and extracting the content out of it. Note, that I'm only extracting text content, in a real world scenario you may want to extract images, tables using semantic parsing techniques.

In [ ]:
import requests
import uuid
from tika import parser 
os.makedirs("./data", exist_ok=True)
file_path = "./data/research_paper.pdf"
if not os.path.exists(file_path):
    print("Downloading research paper...")
    response = requests.get(research_paper_path)
    with open(file_path, "wb") as f:
       f.write(response.content)
else:
    print("Research paper already downloaded.")
raw = parser.from_file(file_path)
content = raw['content']

### Grounding with Bing Search Agent

Grounding helps AI systems understand and reason about the physical world by linking abstract symbols (words, data points) to their real-world meanings and contexts. 
Bing search can be used to get latest information for words, phrases generated by LLMs from internet, this allows you to ground the text generated to facts, links from sources in internet. Before proceeding, complete the following steps

1. Create a Bing Service in Azure, copy the value into the ENV Key/Value pair shown below
2. [Connect](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview) your AI project to the Bing service 



In [ ]:
# create environment variable, change the value to your connection name
os.environ["BING_CONNECTION_NAME "] = "bing-grounding"

bing_connection = project_client.connections.get(
    connection_name="binggrounding"
)

In [ ]:
from azure.ai.projects.models import BingGroundingTool


async def web_ai_agent(query: str) -> str:
    """AIProjectClient agent that uses the Bing search tool to answer questions."""
    # Create a BingGroundingTool instance
    
    conn_id = bing_connection.id
    
    bing = BingGroundingTool(connection_id=conn_id)
    
    project_client = AIProjectClient.from_connection_string(credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"])
    
    
    with project_client:
        
        agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="bing-search-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]

In [ ]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

print("Bing Search Agent created")


In [ ]:
# the below code shows how to use tools in the agent service
# you can also use to call custom function that can get real-time data, perform calculations, and call APIs
async def save_blog_agent(blog_content: str) -> str:
    """AIProjectClient agent that uses the Code Interpreter tool to save blog content."""
    
    print("This is Code Interpreter for Azure AI Agent Service .......")
    
    project_client = AIProjectClient.from_connection_string(credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"])
    
    code_interpreter = CodeInterpreterTool()
    
    agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="blog-save-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    
    # create and execute a run
    
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")
        # print the messages from the agent

    messages = project_client.agents.list_messages(thread_id=thread.id)

    print(f"Messages: {messages}")

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_role("assistant")
    
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        
        
    project_client.agents.delete_agent(agent.id)
    
    print("Deleted agent")

    return "Saved"

In [ ]:
save_blog_content_agent = AssistantAgent(
    name="save_post_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save post content. Respond with 'Saved' to when your post are saved.
    """
)

In [ ]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a linked in post writer, please help me write a linked post based on research paper and bing search content."
    """
)

### Run Orchestration

The below code uses AutoGen to setup a roundrobin group chat. RoundRobinGroupChat is a group chat that invokes agents in a round-robin order. It's useful when you want to call multiple agents in a fixed sequence.

In [ ]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(max_messages=5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination
reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

In [ ]:
await Console(
    reflection_team.run_stream(task=f"""
                    I'm writing a linked in post about a research paper. The content of the research paper is {content}.
                    
                    Extract the following information from the research paper:
                    
                    - Problem/Research Question: What is the research trying to address or investigate? 
                    - Methods/Approach: How did the researchers conduct their study? 
                    - Results/Findings: What did the research uncover? 
                    - Conclusions/Implications: What are the main takeaways and significance of the research? 
                    - Limitations and Future Directions: What
                    
                    Generate a high-engagement Linked In post about the challenges and solutions in the research paper.
                    
                    Extract keywords and use bing search to explain the keywords in the research paper.                    
                    The tone should be authoritative yet engaging. 
                    Follow the Hook → Context → Insights → Engagement Trigger structure. End with a question to spark discussion. Add 3-5 relevant hashtags.
    """)
)  

### Sample Output below


🚀 **Revolutionizing LLM Evaluation: The Rise of RocketEval** 🚀

In an era where large language models (LLMs) are rapidly evolving, the efficacy of their evaluation has become an urgent necessity. Traditional methods often involve costly human evaluations that can compromise privacy and reproducibility. **Enter "RocketEval."**

**Context:** This groundbreaking research presented at ICLR 2025 explores how lightweight LLMs can serve as highly-efficient evaluators through a structured checklist approach. By transitioning the evaluation process from subjective human judgments to a systematic, automated framework, researchers have opened new avenues for cost-effective, scalable assessments that align closely with human preferences.

**Insights:** 
- **Key Findings**: The RocketEval framework demonstrates a remarkable correlation of **0.965** with human evaluations using a lightweight model (Gemma-2-2B), achieving results comparable to powerful models like GPT-4, but at **over a 50-fold** cost reduction! 
- **Innovative Approach**: By reframing evaluations into multi-faceted checklists, the study addresses common pitfalls in LLM assessments: high uncertainty and positional bias. This systematic approach ensures that critical aspects of a model's response are thoroughly evaluated, leading to more reliable outputs.

**Engagement Trigger:** As the landscape of AI continues to shift, how can we ensure that our evaluation methods keep pace with the technology? Do you believe automated systems can fully replace human evaluators, or is there still value in the human touch in this process?

💬 *Let's discuss the future of LLM evaluations!*

#LLM #Automation #AIResearch #DataScience #MachineLearning 

---

**Keywords Explained**:

1. **Large Language Models (LLMs)**: These are complex AI models designed to understand and generate human language. They are foundational in various applications, from chatbots to content generation.

2. **Automated Evaluation**: This refers to the use of technology, especially AI, to assess the performance of models. It aims to provide faster, scalable, and cost-effective assessments compared to traditional methods.

3. **Checklist Grading**: A systematic approach where specific criteria are outlined (checklists) to guide the evaluation process, ensuring comprehensive and consistent assessments.

4. **High Correlation with Human Preferences**: A metric indicating how closely the automated evaluation results match human evaluations, highlighting the effectiveness of the evaluation method.

5. **Positional Bias**: A cognitive bias that affects decision-making based on the order of presented information. Addressing positional bias is crucial in evaluations to enhance fairness and accuracy. 

Feel free to ask if you have more specific questions or need additional information!    
